In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# 1. Configuración inicial y simulación de datos crudos
np.random.seed(42)
num_tickets = 1500
departamentos = ['Ventas', 'Logística', 'Soporte Técnico', 'Facturación']
aspectos_absa = ['Demora en entrega', 'Atención al cliente', 'Falla técnica', 'Error en cobro', 'Duda general']
sentimientos = ['Positivo', 'Neutro', 'Negativo']

# Generador de fechas aleatorias para el último año
def random_date(start_date, end_date):
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    return start_date + timedelta(days=random_number_of_days)

fecha_inicio = datetime(2025, 1, 1)
fecha_fin = datetime(2026, 2, 20)

#Creación del DataFrame principal (Simulando el output de tu pipeline LLM/ABSA)
data = {
    'Ticket_ID': [f'TK-{1000 + i}' for i in range(num_tickets)],
    'Fecha_Creacion': [random_date(fecha_inicio, fecha_fin) for _ in range(num_tickets)],
    'Departamento_ID': np.random.randint(1, len(departamentos) + 1, num_tickets),
    'Aspecto_Detectado': np.random.choice(aspectos_absa, num_tickets, p=[0.3, 0.2, 0.2, 0.15, 0.15]),
    # Simular que Logística y Facturación tienen más sentimientos negativos
    'Sentimiento': np.random.choice(sentimientos, num_tickets, p=[0.15, 0.25, 0.60]),
    'Tiempo_Resolucion_Horas': np.random.randint(1, 120, num_tickets)
}

df_hechos = pd.DataFrame(data)

# Lógica simulada: Si es Negativo y de Logística, tarda más en resolverse
mask_demora = (df_hechos['Sentimiento'] == 'Negativo') & (df_hechos['Aspecto_Detectado'] == 'Demora en entrega')
df_hechos.loc[mask_demora, 'Tiempo_Resolucion_Horas'] += np.random.randint(24, 72, size=mask_demora.sum())

# Calculamos un "Costo Operativo" del ticket basado en las horas de resolución ($15 USD/hora)
df_hechos['Costo_Operativo_USD'] = df_hechos['Tiempo_Resolucion_Horas'] * 15

# Tablas de Dimensiones (Para el Modelo de Estrella en Power BI)
df_dim_departamentos = pd.DataFrame({
    'Departamento_ID': range(1, len(departamentos) + 1),
    'Nombre_Departamento': departamentos
})

df_dim_aspectos = pd.DataFrame({
    'Aspecto_ID': range(1, len(aspectos_absa) + 1),
    'Nombre_Aspecto': aspectos_absa
})

# Mapeamos el Aspecto_ID a la tabla de hechos para normalizarla
aspecto_map = dict(zip(df_dim_aspectos['Nombre_Aspecto'], df_dim_aspectos['Aspecto_ID']))
df_hechos['Aspecto_ID'] = df_hechos['Aspecto_Detectado'].map(aspecto_map)
df_hechos = df_hechos.drop(columns=['Aspecto_Detectado']) # Limpiamos la columna de texto

# 4. Exportación automatizada de los CSV (El input de Power BI)
df_hechos.to_csv('Fact_Tickets.csv', index=False)
df_dim_departamentos.to_csv('Dim_Departamentos.csv', index=False)
df_dim_aspectos.to_csv('Dim_Aspectos.csv', index=False)

print("¡Pipeline ejecutado con éxito! Archivos CSV generados y listos para importar a Power BI.")

¡Pipeline ejecutado con éxito! Archivos CSV generados y listos para importar a Power BI.
